## PROJET GENERATIVE AI

## Carlene TUNGAMWESE

### Problématique : 
Bien que le langage SQL soit devenu une compétence essentielle pour les métiers de l'analyse de données, notamment pour les futurs Data Analysts, il est souvent négligé. De nombreux étudiants atteignent le niveau de Master 2 sans avoir acquis les bases du SQL, ce qui les désavantage face aux exigences du marché du travail. Cette lacune constitue un frein à leur insertion professionnelle, surtout dans des domaines tels que la finance, la banque, et l'assurance, où l’exploitation de bases de données est primordiale.

### Approche : 
Pour combler ce manque, l’idée est de développer un chatbot pédagogique interactif destiné aux étudiants, mais également accessible à tous ceux souhaitant apprendre SQL. Ce chatbot permettra d’apprendre et de pratiquer le langage SQL de manière autonome et ludique, grâce à des exercices progressifs et des explications adaptées.

In [10]:
import os
from dotenv import load_dotenv
import openai


# Charger les variables d'environnement à partir du fichier .env
load_dotenv()

# Obtenir la clé API
openai.api_key = os.getenv('OPENAI_API_KEY')

### 1. Traitement des fichiers Audio

In [ ]:
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import OpenAIWhisperParser
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader
import os
from yt_dlp import YoutubeDL

# Définir le répertoire de sauvegarde
save_dir = "docs/youtube/"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# URLs de vidéos YouTube individuelles
urls = [
    "https://www.youtube.com/watch?v=LRjnjipMe54",
    "https://www.youtube.com/watch?v=bpRirgAiyhw",
    "https://www.youtube.com/watch?v=4Ro69oqZ20s",
    "https://www.youtube.com/watch?v=g9yC8pcdRFc",
    "https://www.youtube.com/watch?v=kawRFZMQ-60",
    "https://www.youtube.com/watch?v=mc4zOX1j7_A",
    "https://www.youtube.com/watch?v=9Aj9X-j1Yys"
]

# Configuration de yt_dlp
ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
    'ffmpeg_location': 'C:/ffmpeg',  # Assurez-vous que ce chemin est correct
    'outtmpl': os.path.join(save_dir, '%(title)s.%(ext)s'),
    'noplaylist': True,
    'ignoreerrors': True,
    'download_archive': os.path.join(save_dir, 'downloaded_videos.txt')
}

# Télécharger et extraire l'audio
with YoutubeDL(ydl_opts) as ydl:
    ydl.download(urls)

# Vérifier l'existence de ffmpeg et imprimer la confirmation
ffmpeg_path = os.path.join(ydl_opts['ffmpeg_location'], 'ffmpeg.exe')
if os.path.exists(ffmpeg_path):
    print("FFmpeg est correctement installé à :", ffmpeg_path)
else:
    print("Le chemin de FFmpeg n'est pas correct :", ffmpeg_path)


In [4]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to c:\users\carlène tungamwese\appdata\local\temp\pip-req-build-onik3vy3
  Resolved https://github.com/openai/whisper.git to commit 25639fc17ddc013d56c594bfbf7644f2185fad84
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git 'C:\Users\carlène Tungamwese\AppData\Local\Temp\pip-req-build-onik3vy3'

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from whisper import load_model

# Charger le modèle Whisper (choisir la taille selon vos besoins: 'tiny', 'base', 'small', 'medium', 'large')
model = load_model("tiny")

# Chemin vers le dossier contenant les fichiers
docs_folder = "docs/youtube"

# Lister tous les fichiers dans le dossier
file_list = os.listdir(docs_folder)

# Parcourir chaque fichier et appliquer Whisper
for file_name in file_list:
    file_path = os.path.join(docs_folder, file_name)
    
    # Transcription avec Whisper
    print(f"Transcription de {file_name} en cours...")
    result = model.transcribe(file_path)
    
    # Afficher ou sauvegarder la transcription
    transcription = result['text']
    print(f"Transcription de {file_name} :\n{transcription}\n")
    
    # Vous pouvez également sauvegarder la transcription dans un fichier texte
    with open(f"{file_name}_transcription.txt", "w", encoding="utf-8") as f:
        f.write(transcription)

#### 2. Etape 1 : Charger les fichiers PDF, .txt et .ipynb

Dans cette étape, nous allons parcourir notre répertoire et charger les fichiers PDF et .ipynb. Nous utiliserons PyPDFLoader pour les PDF et TextLoader pour les fichiers .ipynb et .txt. Ensuite, nous découperons les documents en petits morceaux pour faciliter l'indexation.

In [11]:
import os
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def load_files_from_directory(directory):
    docs = []
    
    # Parcourir tous les fichiers dans le répertoire
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        
        # Charger les fichiers PDF
        if filename.endswith('.pdf'):
            print(f"Chargement du fichier PDF : {file_path}")
            loader = PyPDFLoader(file_path)
            documents = loader.load()

        # Charger les fichiers Jupyter Notebook (.ipynb)
        elif filename.endswith('.ipynb'):
            print(f"Chargement du fichier Jupyter Notebook : {file_path}")
            loader = TextLoader(file_path, encoding='utf-8')
            documents = loader.load()

        # Charger les fichiers texte (.txt)
        elif filename.endswith('.txt'):
            print(f"Chargement du fichier texte : {file_path}")
            loader = TextLoader(file_path, encoding='utf-8')
            documents = loader.load()

        else:
            print(f"Type de fichier non supporté : {filename}")
            continue

        if not documents:
            print(f"Aucun document chargé depuis {filename}")
            continue

        print(f"{len(documents)} document(s) chargé(s) depuis {filename}")
        
        # Diviser le document en morceaux plus petits
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
        split_docs = text_splitter.split_documents(documents)
        docs += split_docs

    return docs

# Utilisation de la fonction avec votre répertoire
directory = r"C:\Users\carlène Tungamwese\OneDrive\Bureau\Nouveau dossier\FORMATION\GENERATIVE AI\docs_SQL"
documents = load_files_from_directory(directory)

# Affichage du nombre de documents chargés et découpés
print(f"Total de documents chargés et découpés : {len(documents)}")


Chargement du fichier PDF : C:\Users\carlène Tungamwese\OneDrive\Bureau\Nouveau dossier\FORMATION\GENERATIVE AI\docs_SQL\0072-cours-langage-sql.pdf
108 document(s) chargé(s) depuis 0072-cours-langage-sql.pdf
Chargement du fichier PDF : C:\Users\carlène Tungamwese\OneDrive\Bureau\Nouveau dossier\FORMATION\GENERATIVE AI\docs_SQL\0172-formation-base-donnees.pdf
50 document(s) chargé(s) depuis 0172-formation-base-donnees.pdf
Chargement du fichier PDF : C:\Users\carlène Tungamwese\OneDrive\Bureau\Nouveau dossier\FORMATION\GENERATIVE AI\docs_SQL\0230-formation-base-donnees-langage-sql.pdf
114 document(s) chargé(s) depuis 0230-formation-base-donnees-langage-sql.pdf
Chargement du fichier PDF : C:\Users\carlène Tungamwese\OneDrive\Bureau\Nouveau dossier\FORMATION\GENERATIVE AI\docs_SQL\0320-programmation-sql-sgbd.pdf
18 document(s) chargé(s) depuis 0320-programmation-sql-sgbd.pdf
Chargement du fichier PDF : C:\Users\carlène Tungamwese\OneDrive\Bureau\Nouveau dossier\FORMATION\GENERATIVE AI\docs

Ignoring wrong pointing object 24 0 (offset 0)


6 document(s) chargé(s) depuis Week+3.pdf
Chargement du fichier PDF : C:\Users\carlène Tungamwese\OneDrive\Bureau\Nouveau dossier\FORMATION\GENERATIVE AI\docs_SQL\Week+4.pdf
14 document(s) chargé(s) depuis Week+4.pdf
Total de documents chargés et découpés : 2559


### Etape 2 : Étape 2 : Indexer les documents avec OpenAI Embeddings

Maintenant que les documents sont chargés et découpés, nous allons les indexer en utilisant des embeddings générés par OpenAI. Cela permettra de rechercher des documents par similarité.

In [12]:
!pip install docarray


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

def index_documents(docs):
    # Créer des embeddings pour les documents
    embeddings = OpenAIEmbeddings()
    
    # Indexer les documents dans une base de données vectorielle
    db = DocArrayInMemorySearch.from_documents(docs, embeddings)
    
    # Créer un récupérateur pour interroger la base de données vectorielle
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 5})
    
    return retriever

# Indexer les documents
retriever = index_documents(documents)

print("Indexation terminée.")


Indexation terminée.


### Etape 3 : Créer la chaîne de questions-réponses conversationnelle

Nous allons maintenant créer la chaîne de questions-réponses (QA) en utilisant ConversationalRetrievalChain, ce qui permettra d'interagir avec les documents de manière conversationnelle, en utilisant GPT-3.5 pour répondre aux questions.

In [14]:
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

def create_qa_chain(retriever, model_name='gpt-3.5-turbo'):
    # Créer une mémoire pour stocker l'historique de la conversation
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key="answer")
    
    # Créer la chaîne de questions-réponses conversationnelle
    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(model_name=model_name, temperature=0), 
        retriever=retriever, 
        memory=memory,
        return_source_documents=True,
        output_key="answer"  # Indique que la clé "answer" est la sortie principale
    )
    
    return qa_chain

# Créer la chaîne QA
qa_chain = create_qa_chain(retriever)

print("Chaîne de questions-réponses créée.")


Chaîne de questions-réponses créée.


### Etape 4 : Poser des questions à la chaîne QA

Enfin, nous allons interagir avec la chaîne QA en posant des questions. La chaîne générera des réponses basées sur les documents indexés et fournira également les documents sources utilisés pour répondre.

In [15]:
def ask_question(qa_chain, question):
    # Poser une question à la chaîne QA
    response = qa_chain({"question": question})
    
    # Afficher la réponse
    print("Réponse : ", response['answer'])
    
    # Afficher les documents sources
    print("\nDocuments sources :")
    for source in response['source_documents']:
        print(source.metadata['source'])

# Exemple d'interaction : poser une question
question = "comment installer SQL et donnez moi un exemple d'une requete"
ask_question(qa_chain, question)

Réponse :  Je suis désolé, mais il semble y avoir une confusion. SQL (Structured Query Language) n'est pas un logiciel à installer, c'est un langage de programmation utilisé pour interagir avec les bases de données relationnelles telles que MySQL, PostgreSQL, SQL Server, etc. 

Pour installer un système de gestion de base de données relationnelle comme MySQL, vous pouvez suivre les instructions spécifiques à chaque système sur leur site officiel. Par exemple, pour MySQL, vous pouvez télécharger MySQL Community Server sur le site officiel de MySQL.

Un exemple de requête SQL simple serait:

```sql
SELECT * FROM nom_de_la_table;
```

Cette requête sélectionne toutes les colonnes de la table spécifiée. Vous pouvez remplacer "nom_de_la_table" par le nom réel de la table que vous souhaitez interroger.

Documents sources :
C:\Users\carlène Tungamwese\OneDrive\Bureau\Nouveau dossier\FORMATION\GENERATIVE AI\docs_SQL\SQL1.txt
C:\Users\carlène Tungamwese\OneDrive\Bureau\Nouveau dossier\FORMATION